# ML Trader

In [1]:
from utils.packages import *
from utils.time_tick import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
warnings.filterwarnings('ignore')

In [2]:
#--------------------------------------------------------------------------------------------------------------------------
# ### Inputs and Parameters

temp_file = 'config/access_keys.yaml'
with open(temp_file) as temp_file:
    config = yaml.load(temp_file)     
    
logging.basicConfig(filename='traderrun.log', level=logging.ERROR)

data['instrument'] = "EUR_USD"
data['account'] = 'oanda_demo_primary'

data['access_token'] = config[data['account']]['token']
data['accountID'] = config[data['account']]['account_id']
data['params'] = {'instruments': data['instrument']}

In [3]:
data['api'] = API(access_token = data['access_token'])
data['request_stream_data'] = pricing.PricingStream(accountID=data['accountID'], params=data['params'])
data['response_stream'] = data['api'].request(data['request_stream_data'])


data["start_ts"] = dt.datetime.strftime(dt.datetime.now(), '%Y-%m-%d-%H-%M')

for data['iter'], data['resp'] in enumerate(data['response_stream']):
    
    
    # Heartbeat check ---------------------------------------------
    if data['resp']['type'] == 'HEARTBEAT': # Heart beat response to keep the data['api'] connection alive (Avoid timeout)
        continue

    # Get time ---------------------------------------------
    data = get_date_time(data)
        
    
    # Time diff check ---------------------------------------------
    if data['time_diff'] > data['max_time_diff']:        
        print(data['time_diff'])
        raise ValueError('Time difference is above limit')
    
    # Get Data ---------------------------------------------
    data = get_prices(data)
    
    print_report(data)
    
    # sema before after loops --------------------------------
    if len(data['sema_tick_list']) < data['sema_len']:
        data =  before_sema(data)

    if len(data['sema_tick_list']) == data['sema_len']:
        data = after_sema(data)                 
    # ----------------------------------------------------------


    # lema before after loops --------------------------------
    if len(data['lema_tick_list']) < data['lema_len']:
        data = before_lema(data)        
        continue

    if len(data['lema_tick_list']) == data['lema_len']:
        data = after_lema(data)                  
    # ----------------------------------------------------------

    


    # Angle of Sema and Lema --------------------------------
    if len(data['sema_angle_list']) < data['angle_len']:
        data = before_angle(data)        
        continue

    if len(data['sema_angle_list']) == data['angle_len']:
        data = after_angle(data)                  
    # ----------------------------------------------------------
    

---------------------------
time : 2021-Mar-29, 06:51:13 (PM)
tick : 1.17725
sema : 1.1772444549763033
sema len : 5
sema list : deque([1.17725, 1.177235, 1.17725, 1.177235, 1.17725])
lema : 1.177240577484278
lema len : 10
lema list : deque([1.177265, 1.17723, 1.17721, 1.177225, 1.177235, 1.17725, 1.177235, 1.17725, 1.177235, 1.17725])
---------------------------


KeyboardInterrupt: 